In [60]:
import re
import pandas as pd
import numpy as np
import os

In [77]:
df = pd.read_csv(r"..\RecipeLinkExtractor\DATA\bbc_food_clean.csv",sep="\t",index_col=0)
df.head()
df['recipe_title'] = None
df['recipe_servings'] = None
df['recipe_ingredients'] = None
df['recipe_steps'] = None

In [81]:
def extractRecipe(row):
    html_path = "..\RecipeLinkExtractor\DATA"
    html_path = html_path + "\\" + row['title'] + ".html"
    
    if os.path.isfile(html_path):
        with open(html_path, "r", encoding='utf-8') as f:
            text= f.read()
            try:
                pattern = "<h1 class=\"gel-trafalgar content-title__text\".*?>(.+?)</h1>"
                title = re.findall(pattern, text)[0]
                row['recipe_title'] = title
            except:
                row['recipe_title'] = None
                
            try:
                pattern = "<p class=\"recipe-metadata__serving\".*?>(.+?)</p>"
                servings = re.findall(pattern, text)[0]
                servings = re.sub("[^0-9]","", servings)
                row['recipe_servings'] = servings
            except:
                row['recipe_servings'] = None
                
            try:    
                pattern_remove = "<[^>]*>"
                pattern = "<ul class=\"recipe-ingredients__list\".*?>(.+?)</ul>"
                x = re.findall(pattern, text)
                ingredients = list()
                for y in x:
                    y = re.sub("</li>","\n",y)
                    ingredients.extend([x.strip() for x in re.sub(pattern_remove,"",y).split("\n") if x != ""])
                row['recipe_ingredients'] = ingredients
            except:
                row['recipe_ingredients'] = None
                
            try:
                pattern = "<ol class=\"recipe-method__list\".*?>(.+?)</ol>"
                pattern_remove = "<[^>]*>"
                steps = list()
                x = re.findall(pattern, text)
                for y in x:
                    y = re.sub("</li>","\n",y)
                    steps.extend([x.strip() for x in re.sub(pattern_remove,"",y).split("\n") if x != ""])
                row['recipe_steps'] = steps
            except:
                row['recipe_steps'] = None
                
            return row
    else:
        print("file neexistuje.",html_path)
        return row

In [82]:
%%time
df = df.apply(lambda x: extractRecipe(x), axis=1)

Wall time: 4.36 s


In [85]:
df.to_csv("RecipeExtracted.csv", sep="\t")